# [Advent of Book](https://milessteele.com/advent-of-book/)

These are the solutions to my 2017 “Advent of Book” Christmas present. (Thanks, Miles!)

Also see the original [Advent of Code](https://adventofcode.com), and [my solutions](https://github.com/osteele/notebooks/blob/master/Advent%20of%20Code%202017.ipynb).

> Santa slipped on the roof and fumbled your link. He's a-ok, but as your link tumbled down the chimney it was **xor**d with the string `hohoho` (repeated). Then it was frozen by a flurry of hexadecimal snowflakes.

Some imports and a utility function:

In [1]:
import operator
from itertools import *
from re import findall

join = ''.join  # join a sequence of strings into a single string

Initialize the puzzle parameters:

In [2]:
CODE = '001b1c1f1b5547401f181f410902091 50701460c070247081840181d070b1d 0c1c400306060b040a451d0d0b0d021 81b010006402a5f3730373037303730 570401013a0a0c0a0d02552821293c4 90306063d0d0b0d0a053b07040d0155 283b30373037303730373037303730'
PAD = 'hohoho'

Convert the codestring from hex. XOR it with the (repeating) pad.

`int` converts the hexadecimal string to an `int`.

`join(CODE.split())` removes spaces. `CODE.replace(' ', '')` would have worked as well. (`split` removes whitespace while this call to `replace` removes only spaces, but they come to the same thing on this string.)

`r'\S{2}'` is a regular expression that matches two non-space characters. `r'\S\S'` would have been equivalent. Since the spaces have already been removed, `r'..'` would have also worked. The use of `\S` is left over from a first attempt, where I didn't realize that some the spaces were *within* hexadecimal pairs.

`cyphertext = […]` would have created a list. `cyphertext = (…)` creates a generator, instead. It therefore never requires memory for all the items in the sequence that it generates. This doesn't matter here – in fact, for such small datasets, it's performance and probably memory usage are probably worse – but it's a nice pattern to use for cryptographic and other data streams.

In [3]:
cyphertext = (int(s, 16) for s in findall(r'\S{2}', join(CODE.split())))
template = join(map(chr, starmap(operator.xor, zip(cyphertext, cycle(map(ord, PAD))))))
template

'https://www.amazon.com/gp/product/kindle-redemption/B0________?kinRedeem=GIFT&kinRedeemToken=GS_____________'

We're not done yet!:

> And that's not all...

> After disentangling the link you will notice 2 gaps (runs of underscores) where letters fell out of the link. To fill the gaps, place the letters back in the index where they belong. To get the index where a letter belongs, sum its associated number with its ASCII value.

Read the part 1 input from the web:

In [4]:
from urllib.request import urlopen

INPUT1 = urlopen('https://milessteele.com/advent-of-book/input1.txt').read().decode()

Create a map from string position to replacement character, as specified in the instructions. The regular expression matches a single character, followed by a single space, followed by the string representations of a (positive or negative) integer. `int` converts the string representation into an `int`; `ord` returns the `ASCII` value of a character (assuming the string is in ASCII, which it is here).

In [5]:
replacements = {int(n) + ord(c): c for c, n in findall(r'(.) (-?\d+)', INPUT1)}
print(replacements)

{96: '5', 97: '5', 98: 'C', 99: 'N', 100: 'W', 101: '3', 102: 'L', 103: '9', 104: 'Z', 105: 'A', 106: '8', 107: '3', 54: '0', 55: 'C', 56: 'K', 57: 'D', 58: 'F', 59: 'E', 60: '9', 61: 'W', 95: 'W'}


Make a new string. It uses the character from the replacement map, for positions where there is one; and the character from the original string, where no replacement is specified.

`print` in Jupyter makes the output a live link.

[`starmap`](https://docs.python.org/3/library/itertools.html#itertools.starmap) is kind of advanced. See "Other Approaches", next, for alternatives that spell out what `starmap` is doing.

In [6]:
print(join(starmap(replacements.get, enumerate(template))))

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


This is indeed a working link. You can't use it to claim my book, though – I've already claimed it!

### Other Approaches

This could also have been written more explicitly without `starmap`:

In [7]:
print(join(replacements.get(i, c) for i, c in enumerate(template)))

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


Or with `[]` instead of `get`:

In [8]:
print(join(replacements[i] if i in replacements else c for i, c in enumerate(template)))

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


Or with an explicit loop over the template:

In [9]:
link = ''
for i, c in enumerate(template):
    if i in replacements:
        link += replacements[i]
    else:
        link += c
print(link)

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


…or a loop over the replacements:

In [10]:
link = list(template)
for i, c in replacements.items():
    link[i] = c
print(join(link))

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


### A note on performance

Consider again:

In [11]:
link = ''
for i, c in enumerate(template):
    if i in replacements:
        link += replacements[i]
    else:
        link += c
print(link)

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


Appending to strings is expensive. It copies the whole string each time. In performance-critical code (which this isn't), we'd make a list of a characters instead:

In [12]:
link = []
for i, c in enumerate(template):
    if i in replacements:
        link.append(replacements[i])
    else:
        link.append(c)
print(join(link))

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


Or, write to a `StringIO`, and then use its value:

In [13]:
from io import StringIO

link = StringIO()
for i, c in enumerate(template):
    if i in replacements:
        link.write(replacements[i])
    else:
        link.write(c)
print(link.getvalue())

https://www.amazon.com/gp/product/kindle-redemption/B00CKDFE9W?kinRedeem=GIFT&kinRedeemToken=GSW55CNW3L9ZA83


## [Part 2](https://milessteele.com/advent-of-book/part2.html)

> The second link is much like the first, but has different fillers for the same gaps. Your puzzle input is a string of symbols. The number of times a symbol occurs in a row, plus a **magic number**, is the index where it belongs. The same symbol might have multiple indices.
>
> For example, if the **magic number** were 2, the link with gaps were `alm___ _o_e!`, and the puzzle input were as below. The answer would be `almost done!`.
>
> `nnnnnnntttodddddss`
>
> The magic number is 53.

Use a regex [back reference](https://www.regular-expressions.info/backref.html) to match a run of repeated characters. `(.)` matches any character. It's the second `(` in the regular expression, so `\2` refers back to it. The outermost `(…)` matches the whole expression, so its length is the number of repetitions.

In [14]:
INPUT2 = 'SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK777777777777777777777777777777777777777777777555VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV6GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSBBBBBBWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW2222222222222222222222222222222222222222222222SSSSS88888888TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT555533333333333333333333333333333333333333333333333333333YYQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ666666666666666666666666666666666666666666HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH4444444'
MAGIC = 53

replacements = {len(reps) + MAGIC: c for reps, c in findall(r'((.)(?:\2*))', INPUT2)}
print(join(starmap(replacements.get, enumerate(template))))

https://www.amazon.com/gp/product/kindle-redemption/B06Y55SB48?kinRedeem=GIFT&kinRedeemToken=GS6VS72WSTQKH3G


### Alternatives

A simpler regular expression (it doesn't construct a group that matches both the initial character together with all its repetitions), with (slightly) more complicated code:

In [15]:
replacements = {1 + len(reps) + MAGIC: c for c, reps in findall(r'(.)(\1*)', INPUT2)}
print(join(starmap(replacements.get, enumerate(template))))

https://www.amazon.com/gp/product/kindle-redemption/B06Y55SB48?kinRedeem=GIFT&kinRedeemToken=GS6VS72WSTQKH3G


#### Without regular expressions

In [16]:
replacements = dict()
c0, reps = None, None
for c in INPUT2:
    if c != c0:
        if reps:
            replacements[MAGIC + reps] = c0
        c0, reps = c, 0
    reps += 1
if reps:
    replacements[MAGIC + reps] = c0

print(join(starmap(replacements.get, enumerate(template))))

https://www.amazon.com/gp/product/kindle-redemption/B06Y55SB48?kinRedeem=GIFT&kinRedeemToken=GS6VS72WSTQKH3G


If we're going imperative anyway, we might as well go full imperative and update the string directly:

In [17]:
link = list(template)
c0, reps = None, None
for c in INPUT2:
    if c != c0:
        if reps:
            link[MAGIC + reps] = c0
        c0, reps = c, 0
    reps += 1
if reps:
    link[MAGIC + reps] = c0

print(join(link))

https://www.amazon.com/gp/product/kindle-redemption/B06Y55SB48?kinRedeem=GIFT&kinRedeemToken=GS6VS72WSTQKH3G


That duplicate `if reps:` clause is annoying. Replace it with a [sentinel](https://en.wikipedia.org/wiki/Sentinel_value).

In [18]:
link = list(template)
assert '!' not in INPUT2
for c in INPUT2 + '!':
    if c != c0:
        if reps:
            link[MAGIC + reps] = c0
        c0, reps = c, 0
    reps += 1

print(join(link))

https://www.amazon.com/gp/product/kindle-redemption/B06Y55SB48?kinRedeem=GIFT&kinRedeemToken=GS6VS72WSTQKH3G


#### With an iterator

We could also use [`groupby`](https://docs.python.org/3/library/itertools.html#itertools.groupby) from the [itertools module](https://docs.python.org/3/library/itertools.html).

In [19]:
def iter_len(iterable):
    "Return the length of an iterable."
    return sum(1 for _ in iterable)

replacements = {iter_len(g) + MAGIC: c for c, g in groupby(INPUT2)}
print(join(starmap(replacements.get, enumerate(template))))

https://www.amazon.com/gp/product/kindle-redemption/B06Y55SB48?kinRedeem=GIFT&kinRedeemToken=GS6VS72WSTQKH3G
